# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.externals import joblib
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.model import Model
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment

from azureml.data.dataset_factory import TabularDatasetFactory

import requests
import json


## Compute Target

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# choose a name for experiment
experiment_name = 'heart_exp_aml'
project_folder = './'

experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "heart-data"
description_text = "Heart desease data for capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = "https://github.com/koettet/Capstone-Project/blob/9bc802ff0216f3eb846a883f358593ce7b28c03d/starter_file/heart.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# Show details of run
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()

best_run
fitted_model

# print(best_run)

# print(fitted_model)

# Print model steps
# print(fitted_model.steps)   

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

model_name = best_run.properties['model_name']
description = 'Best model to of AML to predict heart conditions'

model = remote_run.register_model(model_name = model_name,
                           tags = None,
                           description = description)

print(remote_run.model_id)

inference_config = InferenceConfig(environment=best_run.get_environment(), entry_script=script_file_name)

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1,
    tags = {'type': 'automl-classification'},
    description = 'Service to classify heart data.'
)

service = Model.deploy(
    ws,
    "heart-service",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)
print(service.state)

In [ ]:

data = {

            "data": [ {

                "age": 51,

                "sex": 0,

                "cp": 1,

                "trestbps": 125,

                "chol": 301,

                "fbs": 1,

                "restecg": 1,

                "thalach": 160,

                "exang": 0,

                "oldpeak": 0.3,

                "slope": 2,

                "ca": 0,

                "thal": 2

         }, {

                "age": 57,

                "sex": 1,

                "cp": 0,

                "trestbps": 98,

                "chol": 226,

                "fbs": 0,

                "restecg": 1,

                "thalach": 150,

                "exang": 0,

                "oldpeak": 2.5,

                "slope": 2,

                "ca": 1,

                "thal": 3
        },],
}

# Convert to JSON string
samp = json.dumps(data)

resp = service.run(input_data = samp)
resp

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests

service = Webservice(workspace=ws, name="heart-service")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs
# data = pd.DataFrame(
#     {"age": pd.Series([51, 57], dtype="int64"), 
#      "sec": pd.Series([0, 1], dtype="int64"), 
#      "cp": pd.Series([1, 0], dtype="int64"), 
#      "trestbps": pd.Series([125, 98], dtype="int64"), 
#      "chol": pd.Series([301, 226], dtype="int64"), 
#      "fbs": pd.Series([1, 0], dtype="int64"), 
#      "restecg": pd.Series([1, 1], dtype="int64"), 
#      "thalach": pd.Series([160, 150], dtype="int64"), 
#      "exang": pd.Series([0, 0], dtype="int64"), 
#      "oldpeak": pd.Series([0.3, 2.5], dtype="float64"), 
#      "ca": pd.Series([0, 1], dtype="int64"), 
#      "thal": pd.Series([2, 3], dtype="int64")}


data = {

            "data": [ {

                "age": 51,

                "sex": 0,

                "cp": 1,

                "trestbps": 125,

                "chol": 301,

                "fbs": 1,

                "restecg": 1,

                "thalach": 160,

                "exang": 0,

                "oldpeak": 0.3,

                "slope": 2,

                "ca": 0,

                "thal": 2

         }, {

                "age": 57,

                "sex": 1,

                "cp": 0,

                "trestbps": 98,

                "chol": 226,

                "fbs": 0,

                "restecg": 1,

                "thalach": 150,

                "exang": 0,

                "oldpeak": 2.5,

                "slope": 2,

                "ca": 1,

                "thal": 3
        },],
}

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
# resp = requests.post(scoring_uri, input_data, headers=headers)
resp = service.run(input_data)
print(resp)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# load existing web service
# service = Webservice(name='heart-service', workspace=ws)
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)
    
# service.delete()
# compute_target.delete()
# model.delete()  

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
